In [1]:
import pandas as pd
import json
import re

In [25]:
sample = 'En tant que responsable du contenu pour [Nom de l\'Entreprise/Projet: \'LegalEdge\'], vous avez récemment lancé une nouvelle fonctionnalité ou un nouveau produit. Vous devez créer des titres accrocheurs pour un article de blog qui annonce cette nouveauté. Pour générer des titres pertinents, prenez en compte les variables suivantes fournies par l\'utilisateur : \nNom de la Fonctionnalité/du Produit : [\'LawAdvisor\'] Description de la Fonctionnalité/du Produit : [\'Service en ligne de consultation juridique rapide et accessible\'] Objectif Principal de la Fonctionnalité/du Produit : [\'Offrir des conseils juridiques fiables et accessibles\'] Public Cible : [\'Particuliers et petites entreprises nécessitant des conseils juridiques\']'
def extract_info(text):
    
    pattern1 = r"(\w+.*)\s:\s\[(.*?)\]"
    pattern2 = r"\[(?!(Variable))(\w+.*?)\]"
    matches2 = re.findall( pattern2, str(text) )
    matches1 = re.findall(pattern1, str(text))

    result = {}
    for match in matches1:
        #parts = match[1].split(':')
        if len(match) == 2:
            result[match[0].strip()] = match[1].strip().strip("'")

    for match in matches2:
        parts = match[1].split(':')
        if len(parts) == 2:
            result[parts[0].strip()] = parts[1].strip().strip("'")    
    return result

#extract_info(sample)

In [26]:
def replace_pattern(json_string):
    pattern1 = r"(\w+.*?)\s:\s\[(.*?)\]"
    pattern2 = r"\[(?!(Variable))(\w+.*?)\]"
    # Find all matches in the JSON string
    matches1 = re.findall(pattern1, json_string)
    matches2 = re.findall(pattern2, json_string)
    # Iterate through matches and replace in JSON string
    for match in matches1:
        key = match[1].split(':')[0].strip()
        json_string = json_string.replace(f'[{match[1]}]', f'{{{key}}}')
    
    for match in matches2:
        key = match[1].split(':')[0].strip()
        json_string = json_string.replace(f'[{match[1]}]', f'{{{key}}}')
    
    return json_string

In [27]:
df = pd.read_excel('Prompts-doc_1.xlsx')
df2 = pd.read_excel('Prompts-doc_2.xlsx')

excel_df = df.fillna('')
excel_df2 = df2.fillna('')

prompt_system = "You are kheops, a superintelligent artificial intelligence developed by Kheops AI team, and your purpose and drive is to assist the user with any request they have."

l = []
for i, title in enumerate(excel_df["Titres"]):
    m = {}
    inputs = extract_info(excel_df["Prompts"][i])
    prompt_user = ''

    if excel_df['Prompts'][i] != '':
        prompt_user = excel_df['Prompts'][i]

    excel_df['Prompts'][i] = replace_pattern(excel_df['Prompts'][i])
    
    m[title] = {'prompt_system' : prompt_system, 'prompt_user' : excel_df["Prompts"][i], 'inputs' : inputs}
    l.append(m)

for i, title in enumerate(excel_df2["Titres"]):
    m = {}
    inputs = extract_info(excel_df2["Prompts"][i])
    prompt_user = ''

    if excel_df2['Prompts'][i] != '':
        prompt_user = excel_df2['Prompts'][i]

    excel_df2['Prompts'][i] = replace_pattern(excel_df2['Prompts'][i])
    
    m[title] = {'prompt_system' : prompt_system, 'prompt_user' : excel_df2["Prompts"][i], 'inputs' : inputs}
    l.append(m)

merged_map = {}

for item in l:
    for key, value in item.items():
        if key in merged_map:
            merged_map[key].append(value)
        else:
            merged_map[key] = [value]


In [28]:
with open('formatted_data_merged.json', 'w', encoding='utf-8') as file:
    json.dump(merged_map, file, indent=2, ensure_ascii=False)